# Importing necessary libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Uploading data and selecting variables to use

#
I have uploaded Fraud.csv .By analysing the data I found that I have to predict 2nd last column on the basis of 1st 9 columns. Also I found that data in the 4th and 7th column are names only which does not affect our model of prediction so I decided to choose 1-9 columns except 4th and 7th as X and 10th column as y.  

In [2]:
dataset = pd.read_csv('Fraud.csv')
X = dataset.iloc[:,[0,1,2,4,5,7,8]].values
y = dataset.iloc[:, -2].values

# Data cleaning including missing values, outliers and multi-collinearity

# 
Since data given is too large so we can't check every column but there may be missing values in the data so I have imported simple imputer from sklearn.impute and replaced missing data by mean of data in that column.

In [3]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(X[:, 2:7])
X[:, 2:7] = imputer.transform(X[:, 2:7])

I found that data in 2nd column was a categorical data so I decided to replace them by some binary code so I imported column transformer from sklearn.compose. 

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

I devided the given data in the ratio of 4:1 anad named 4/5 of it as X_train and y_train and the other part as X_test nad y_test.

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

I found that datasets which will be used for training and testing were in different ranges so I decided to bring them in range of (-3,3).So I imported standardscalar from sklearn.preprocessing.

In [6]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train[:, 6:] = sc.fit_transform(X_train[:, 6:])
X_test[:, 6:] = sc.transform(X_test[:, 6:])

# Choosing a model for prediction and describing it
I have compared acccuracies of different classification models and found that XGBoost classifier has highest accuracy.
# XGBoost
XGBoost is an implementation of Gradient Boosted decision trees. XGBoost models majorly dominate in many Kaggle Competitions.

In this algorithm, decision trees are created in sequential form. Weights play an important role in XGBoost. Weights are assigned to all the independent variables which are then fed into the decision tree which predicts results. The weight of variables predicted wrong by the tree is increased and these variables are then fed to the second decision tree. These individual classifiers/predictors then ensemble to give a strong and more precise model. It can work on regression, classification, ranking, and user-defined prediction problems.

In [8]:
from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

# Demonstration of performance
We have not used testing data in the training so now we will make prdiction for X_test and then compare y_pred with y_test to compute accuracy and hence we will find performance of the given model

In [9]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1270840      37]
 [    205    1442]]


0.9998098267694755

In [10]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 99.98 %
Standard Deviation: 0.00 %


In [11]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])

# key factors that predict fraudulent customer
Step,Type,Amount,Old balnce ,New balance thses are the features used in the given model which predicts fraudaulent customer. yes ,These factors make sense as thse features will show some abnormal behaviour when the customer is fraud.

# What kind of prevention should be adopted while company update its infrastructure?
Company should see features of all customers and use these type of testing models to predict whether the customer is fraudaulent or not.
# Assuming these actions have been implemented, how would you determine if they work?
We will again check the accuracy of our model to see whether it is working or not in the future if fraudaulent features changes then we may need to change model and try some different model for prediction.